<a href="https://colab.research.google.com/github/misrori/scraping_2024/blob/main/session_4_coingecko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coingecko api

https://www.coingecko.com/api/documentation

In [ ]:
!pip install scrapethat

In [ ]:
import pandas as pd

In [ ]:
# Find the top 10 trending coins and compare the price change of the last month

import requests

url = "https://api.coingecko.com/api/v3/search/trending"

headers = {
    "accept": "application/json",
    "x-cg-demo-api-key": "CG-......"
}

response = requests.get(url, headers=headers)

t = response.json()


In [ ]:
t

NameError: name 't' is not defined

In [ ]:
trending = pd.DataFrame(list(map( lambda x: {'name': x['item']['name'], 'id': x['item']['slug']}, t['coins'])))

In [ ]:
# get olhc data of trending tokens
def get_one_token(token_id):


    try:
        url = f"https://api.coingecko.com/api/v3/coins/{token_id}/ohlc?vs_currency=usd&days=30"

        headers = {
            "accept": "application/json",
            "x-cg-demo-api-key": "CG-........"
        }

        response = requests.get(url, headers=headers)
        df = pd.DataFrame(response.json())
        df.columns = ['date', 'open', 'high', 'low', 'close']
        df['date'] = pd.to_datetime(df['date'], unit='ms')
        df['token'] = token_id
        first_close = df['close'].iloc[0]
        df['change'] = (( df['close']  / first_close ) -1) * 100

        return (df)
    except:
        return None


one_token = get_one_token(trending.loc[3, 'id'])
one_token


,date,open,high,low,close,token,change
0,2024-10-27 08:00:00,0.094389,0.094648,0.093924,0.093981,stellar,0.000000
1,2024-10-27 12:00:00,0.093923,0.093942,0.093372,0.093475,stellar,-0.538407
2,2024-10-27 16:00:00,0.093459,0.094239,0.093459,0.094174,stellar,0.205361
3,2024-10-27 20:00:00,0.094219,0.094262,0.093535,0.093869,stellar,-0.119173
4,2024-10-28 00:00:00,0.093910,0.094514,0.093776,0.094113,stellar,0.140454
...,...,...,...,...,...,...,...
175,2024-11-25 12:00:00,0.521228,0.543867,0.518329,0.518329,stellar,451.525308
176,2024-11-25 16:00:00,0.516505,0.516505,0.487691,0.500016,stellar,432.039455
177,2024-11-25 20:00:00,0.499471,0.522799,0.498921,0.516699,stellar,449.790915
178,2024-11-26 00:00:00,0.517547,0.519595,0.480630,0.484235,stellar,415.247763


In [ ]:
from tqdm import tqdm
all_df = pd.concat(map(get_one_token, tqdm(trending['id'])))

100%|██████████| 15/15 [00:02<00:00,  6.76it/s]


In [ ]:
all_df

,date,open,high,low,close,token,change
0,2024-10-27 08:00:00,2480.64,2480.89,2475.39,2478.66,ethereum,0.000000
1,2024-10-27 12:00:00,2478.14,2479.88,2463.20,2466.74,ethereum,-0.480905
2,2024-10-27 16:00:00,2466.85,2499.21,2466.85,2495.16,ethereum,0.665682
3,2024-10-27 20:00:00,2498.01,2503.78,2483.54,2489.91,ethereum,0.453874
4,2024-10-28 00:00:00,2491.18,2522.28,2488.17,2505.50,ethereum,1.082843
...,...,...,...,...,...,...,...
175,2024-11-25 12:00:00,98148.00,98882.00,98095.00,98259.00,bitcoin,46.390846
176,2024-11-25 16:00:00,98295.00,98329.00,94824.00,96413.00,bitcoin,43.640589
177,2024-11-25 20:00:00,96043.00,96294.00,94532.00,94768.00,bitcoin,41.189792
178,2024-11-26 00:00:00,94624.00,95261.00,92775.00,93005.00,bitcoin,38.563192


In [ ]:
import plotly.express as px

fig = px.line(
    all_df,
    x='date',
    y='change',
    color='token',
    template='plotly_white',
)

# Update layout for better visuals
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Percentage Change (%)',
    legend_title='Token',
    title='Percentage Change Over Time by Token',

)

fig.show()